Abdul Rahim - k21-3951

### Task: Design a Variational Quantum Classifier (VQC) model for the specified datasets using the details provided below.


Utilize the following datasets:
- Statlog Heart : https://archive.ics.uci.edu/dataset/145/statlog+heart
- Ionosphere : https://archive.ics.uci.edu/dataset/52/ionosphere

Data Split:
- Training: 70%
- Validation: 10%
- Testing: 20%

Feature Maps:
- Use ZZFeatureMap(..., reps=2)
- Use PauliFeatureMap(..., reps=2, paulis=["Z", "YY"], entanglement="full")

### Design a custom variational quantum circuit (ansatz) from scratch 
- for both feature map and 
- for both datasets (total 4), 

rather than using predefined ansatz options.

Evaluation Metrics:
- Use Accuracy, Precision, Recall, and F1 Score to evaluate the model's performance.


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from qiskit.circuit.library import ZZFeatureMap, PauliFeatureMap
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from qiskit_aer import Aer
from qiskit.primitives import Sampler 
from qiskit_algorithms.optimizers import SLSQP 
from qiskit_machine_learning.algorithms import VQC
from qiskit_machine_learning.circuit.library import RawFeatureVector

In [ ]:
statlog_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/heart/heart.dat"
statlog_columns = [
    'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 
    'oldpeak', 'slope', 'ca', 'thal', 'target'
]
statlog_data = pd.read_csv(statlog_url, delim_whitespace=True, header=None, names=statlog_columns)

ionosphere_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data"
ionosphere_columns = [f'feature_{i}' for i in range(34)] + ['target']
ionosphere_data = pd.read_csv(ionosphere_url, header=None, names=ionosphere_columns)

In [11]:
# categorical labels (target) --> binary values
statlog_data['target'] = statlog_data['target'].apply(lambda x: 1 if x == 1 else 0)
ionosphere_data['target'] = ionosphere_data['target'].apply(lambda x: 1 if x == 'g' else 0)

# separate features (X) and labels (y) for both datasets
X_statlog = statlog_data.drop('target', axis=1)
y_statlog = statlog_data['target']

X_ionosphere = ionosphere_data.drop('target', axis=1)
y_ionosphere = ionosphere_data['target']

# standardize the features (mean=0, std=1)
scaler = StandardScaler()
X_statlog_scaled = scaler.fit_transform(X_statlog)
X_ionosphere_scaled = scaler.fit_transform(X_ionosphere)

In [12]:
def split_data(X, y):
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, random_state=42)  # 0.125 * 80% = 10%
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [13]:
X_train_statlog, X_val_statlog, X_test_statlog, y_train_statlog, y_val_statlog, y_test_statlog = split_data(X_statlog_scaled, y_statlog)

X_train_ionosphere, X_val_ionosphere, X_test_ionosphere, y_train_ionosphere, y_val_ionosphere, y_test_ionosphere = split_data(X_ionosphere_scaled, y_ionosphere)


In [ ]:
print("Statlog Heart Dataset:")
print("Training data shape:", X_train_statlog.shape)
print("Validation data shape:", X_val_statlog.shape)
print("Testing data shape:", X_test_statlog.shape)

print("\nIonosphere Dataset:")
print("Training data shape:", X_train_ionosphere.shape)
print("Validation data shape:", X_val_ionosphere.shape)
print("Testing data shape:", X_test_ionosphere.shape)

In [15]:
def get_zz_feature_map(num_qubits):
    zz_feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=2)
    return zz_feature_map

def get_pauli_feature_map(num_qubits):
    pauli_feature_map = PauliFeatureMap(feature_dimension=num_qubits, 
                                        reps=2, 
                                        paulis=["Z", "YY"], 
                                        entanglement='full')
    return pauli_feature_map


In [16]:
num_qubits_statlog = X_statlog_scaled.shape[1]  # Statlog Heart
num_qubits_ionosphere = X_ionosphere_scaled.shape[1]  # Ionosphere

In [ ]:
zz_map_statlog = get_zz_feature_map(num_qubits_statlog)
pauli_map_statlog = get_pauli_feature_map(num_qubits_statlog)

zz_map_ionosphere = get_zz_feature_map(num_qubits_ionosphere)
pauli_map_ionosphere = get_pauli_feature_map(num_qubits_ionosphere)

print("ZZFeatureMap for Statlog Heart dataset:")
print(zz_map_statlog)

print("\nPauliFeatureMap for Statlog Heart dataset:")
print(pauli_map_statlog)

print("\nZZFeatureMap for Ionosphere dataset:")
print(zz_map_ionosphere)

print("\nPauliFeatureMap for Ionosphere dataset:")
print(pauli_map_ionosphere)

### Ansatz 1: Statlog Heart Dataset with ZZFeatureMap

In [ ]:
def custom_ansatz_statlog_zz(num_qubits):
    qc = QuantumCircuit(num_qubits)
    
    params_rx = [Parameter(f'θ_rx_{i}') for i in range(num_qubits)]
    params_rz = [Parameter(f'θ_rz_{i}') for i in range(num_qubits)]
    
    for i in range(num_qubits):
        qc.rx(params_rx[i], i)
        qc.rz(params_rz[i], i)
    
    for i in range(num_qubits - 1):
        qc.cz(i, i + 1)
    
    return qc

num_qubits_statlog = 13  # no of features in Statlog Heart dataset
statlog_zz_ansatz = custom_ansatz_statlog_zz(num_qubits_statlog)
print("Custom Ansatz for Statlog Heart with ZZFeatureMap:")
print(statlog_zz_ansatz)


### Ansatz 2: Statlog Heart Dataset with PauliFeatureMap

In [ ]:
def custom_ansatz_statlog_pauli(num_qubits):
    qc = QuantumCircuit(num_qubits)
    
    params_ry = [Parameter(f'θ_ry_{i}') for i in range(num_qubits)]
    params_rz = [Parameter(f'θ_rz_{i}') for i in range(num_qubits)]
    
    for i in range(num_qubits):
        qc.ry(params_ry[i], i)
        qc.rz(params_rz[i], i)
    
    for i in range(num_qubits):
        for j in range(i+1, num_qubits):
            qc.cx(i, j)
    
    return qc

statlog_pauli_ansatz = custom_ansatz_statlog_pauli(num_qubits_statlog)
print("\nCustom Ansatz for Statlog Heart with PauliFeatureMap:")
print(statlog_pauli_ansatz)


### Ansatz 3: Ionosphere Dataset with ZZFeatureMap

In [ ]:
def custom_ansatz_ionosphere_zz(num_qubits):
    qc = QuantumCircuit(num_qubits)
    
    params_rx = [Parameter(f'θ_rx_{i}') for i in range(num_qubits)]
    params_ry = [Parameter(f'θ_ry_{i}') for i in range(num_qubits)]
    
    for i in range(num_qubits):
        qc.rx(params_rx[i], i)
        qc.ry(params_ry[i], i)
    
    for i in range(num_qubits - 1):
        qc.cz(i, i + 1)
    
    return qc

num_qubits_ionosphere = 34  
ionosphere_zz_ansatz = custom_ansatz_ionosphere_zz(num_qubits_ionosphere)
print("\nCustom Ansatz for Ionosphere with ZZFeatureMap:")
print(ionosphere_zz_ansatz)


### Ansatz 4: Ionosphere Dataset with PauliFeatureMap

In [ ]:
def custom_ansatz_ionosphere_pauli(num_qubits):
    qc = QuantumCircuit(num_qubits)
    
    params_rx = [Parameter(f'θ_rx_{i}') for i in range(num_qubits)]
    params_ry = [Parameter(f'θ_ry_{i}') for i in range(num_qubits)]
    
    for i in range(num_qubits):
        qc.rx(params_rx[i], i)
        qc.ry(params_ry[i], i)
    
    for i in range(num_qubits):
        for j in range(i+1, num_qubits):
            qc.cx(i, j)
    
    return qc

ionosphere_pauli_ansatz = custom_ansatz_ionosphere_pauli(num_qubits_ionosphere)
print("\nCustom Ansatz for Ionosphere with PauliFeatureMap:")
print(ionosphere_pauli_ansatz)


### Now training the VQC

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X_statlog_scaled, statlog_data['target'], test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42)

In [37]:
backend = Aer.get_backend('aer_simulator')

def create_vqc(feature_map, ansatz):
    vqc = VQC(feature_map=feature_map, ansatz=ansatz, optimizer=SLSQP(maxiter=100), backend=backend)
    return vqc

num_features = X_train.shape[1]
zz_map_statlog = ZZFeatureMap(feature_dimension=num_features, reps=2)
pauli_map_statlog = PauliFeatureMap(feature_dimension=num_features, reps=2, paulis=["Z", "YY"], entanglement="full")

def custom_ansatz(num_qubits):
    circuit = QuantumCircuit(num_qubits)
    for qubit in range(num_qubits):
        circuit.rx(np.random.uniform(0, 2 * np.pi), qubit)  
    circuit = circuit.compose(QuantumCircuit(num_qubits).cx(0, 1))  

In [ ]:
statlog_ansatz = custom_ansatz(num_features)

vqc_model = create_vqc(zz_map_statlog, statlog_ansatz)

vqc_model.fit(X_train, y_train)

y_pred = vqc_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


### Thank You
